In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from tqdm import tqdm
sys.path.append('/home/shyam/BareBonesAI/Deep Learning/')
from nn_optimization_methods import SGD
from layers.Dense import Dense
from layers.Input import Input
sys.path.append('../tools/')
import tools

%matplotlib inline

## DNN from scratch

approximate sin function from random values.

In [2]:
x = np.linspace(-2, 2, 1000)

In [3]:
y = np.sin(x)**2

In [4]:
df = pd.DataFrame({'x':x,'y':y})

In [5]:
class DNN():
    
    """
    Deep Neural Network Template for regression and classification tasks
    Parameters:
        X: numpy array() data matrix
        y: numpy array() response variables, must be numeric
        output: numpy array() outputs
        outlayer = output layer
        head = input layer
    
    Use like sequential model from Keras:
        Ex: add a dense layer with 200 neurons and a sigmoid activation function:
                dnn.add("dense","sigmoid",20)
    """
    def __init__(self,X,y):
            
        self.X = X
        self.y = y
        self.output = 0
        self.outlayer = Input(self.X)
        self.head = self.outlayer
    
    def add(self,layertype,activation,neurons):
        
        if layertype == "dense":
                layer = Dense(self.outlayer,neurons,activation)
                self.outlayer.setNext(layer)
                self.outlayer = layer
            
    def foward(self,inputs=None):
        
        if np.all(inputs != None):
            
            self.head.update(inputs)
            
        layer = self.head
        
        while np.all(layer.next != None):
            
            layer.foward()
            layer = layer.getNext()
        
        layer.foward()
        return layer.output
        
    def backward_pass(self,predictions,y,loss="MSE"):
        
        cache = 0
        gradients = []
        
        if loss == "MSE":
            
            layer = self.outlayer
            cache = np.array(np.sum(predictions - y))
            derivative = layer.getPrev().dot(cache*(layer.deriv),True)
            gradients.append(derivative)
            layer = layer.getPrev()
            
            while np.all(layer.getPrev() != None):
                cache = (cache.dot(layer.getNext().getWeights().T))*layer.deriv
                gradients.append(layer.getPrev().dot(cache,True))
                layer = layer.getPrev()
            
            return gradients[::-1]
        
    def train(self,X,y,optimizer=SGD):
        
        optimizer(self,X,y)
        
    def predict(self,X):
        
        return self.foward(X)
        
    def __str__(self):
        
        string = "----------------------------- Model -----------------------------" + '\n'
        layer = self.head
        
        while np.all(layer != None):
            string += layer.__str__()
            layer = layer.getNext()
            
        return string
    

In [6]:
x = x.reshape(x.shape[0],1)
dnn = DNN(x,y)
dnn.add("dense","sigmoid",20)
dnn.add("dense","sigmoid",10)
dnn.add("dense","sigmoid",400)
dnn.add("dense","linear",1)
print(dnn)

----------------------------- Model -----------------------------
Input: shape: (None, 1)
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (1, 20), output shape: (None, 20), parameters: 20
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (20, 10), output shape: (None, 10), parameters: 200
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (10, 400), output shape: (None, 400), parameters: 4000
-----------------------------------------------------------------
Dense: activation: linear, weight shape: (400, 1), output shape: (None, 1), parameters: 400
-----------------------------------------------------------------



In [7]:
dnn.train(x,y)

  1%|          | 1/100 [00:00<01:30,  1.10it/s]

MSE: 0.5021254670090168


  2%|▏         | 2/100 [00:01<01:32,  1.06it/s]

MSE: 0.13665409791725872


  3%|▎         | 3/100 [00:03<01:54,  1.18s/it]

MSE: 0.13608357476571326


  4%|▍         | 4/100 [00:04<01:55,  1.20s/it]

MSE: 0.1355210338660618


KeyboardInterrupt: 

In [ ]:
predictions = dnn.predict(x)

In [ ]:
predictions

In [ ]:
newd = pd.DataFrame({'x':x,'y':m.output.reshape(m.output.shape[0],)})

Predicted after 10000 epochs

In [ ]:
sns.lmplot(x='x', y='y',data=newd,
           fit_reg=False, # No regression line
          )  

Original sin wave

In [ ]:
sns.lmplot(x='x', y='y',data=df,
           fit_reg=False, # No regression line
          )   